# Test Matrix
Report the test matrix from the ReFrame config.

This is very `hpc-tests` specific, as it assumes tests map 1:1 to environments.

Note although it shows whether an environment has output/perflogs, this does not show whether individual benchmarks defined in a test have results.

In [8]:
import sys, os, json, pprint
import pandas as pd

CFG_PATH = '../reframe_config.py'

def read_config(path):
    config = {}
    with open(path) as cfgf:
        exec(cfgf.read(), {}, config)
    return config['site_configuration']

matrix = {} # environment (= test) name -> ['system1:partition1', ...]
all_sys_parts = [] # ['system1:partition1', ...]

cfg = read_config(CFG_PATH)
for system in cfg['systems']:
    sys_name = system['name']
    for partition in system['partitions']:
        part_name = partition['name']
        #print(' ', part_name)
        sys_part = '%s:%s' % (sys_name, part_name)
        all_sys_parts.append(sys_part)
        for env_name in sorted(partition['environs']):
            envs = [e for e in cfg['environments'] if e['name'] == env_name]
            sys_envs = [e for e in envs if sys_part in e.get('target_systems', [])]
            assert len(sys_envs) < 2, sys_envs
            default_envs = [e for e in envs if not 'target_systems' in e]
            assert len(default_envs) < 2, default_env

            if env_name not in matrix:
                matrix[env_name] = {}
            if sys_envs or default_envs:
                matrix[env_name][sys_part] = 'T'
                if os.path.exists(os.path.join('..', 'output', sys_name, part_name, env_name)):
                    matrix[env_name][sys_part] += 'O'
                if os.path.exists(os.path.join('..', 'output', sys_name, part_name, env_name)):
                    matrix[env_name][sys_part] += 'P'
            else:
                raise ValueError('no definition of %s found for %s' % (sys_part, env_name))
                
# produce dataframe for neat display:

print('Key: [T]est defined, [O]utput found, [P]erflog found')
df = pd.DataFrame(matrix).fillna('-')
display(df)

Key: [T]est defined, [O]utput found, [P]erflog found


,castep,imb,intel-hpl,nxnlatbw,sysinfo,wrf,gromacs,omb,openfoam,intel-hpcg,cp2k,hpl
csd3:cclake-ib-icc19-impi19-ucx,TOP,TOP,TOP,TOP,TOP,TOP,-,-,-,-,-,-
csd3:cclake-roce-icc19-impi19-ucx,TOP,TOP,TOP,TOP,-,TOP,-,-,-,-,-,-
csd3:cclake-ib-gcc9-ompi3-ucx,-,TOP,-,TOP,-,-,TOP,TOP,TOP,-,-,-
csd3:cclake-roce-gcc9-ompi3-ucx,-,TOP,-,TOP,-,-,TOP,TOP,TOP,-,-,-
alaska:ib-gcc9-impi-verbs,-,TOP,T,-,-,-,-,TOP,-,T,-,-
alaska:roce-gcc9-impi-verbs,-,TOP,T,-,-,-,-,TOP,-,T,-,-
alaska:ib-gcc9-openmpi4-ucx,-,TOP,-,TOP,TOP,-,TOP,TOP,TOP,-,TOP,TOP
alaska:roce-gcc9-openmpi4-ucx,-,TOP,-,TOP,-,-,TOP,TOP,TOP,-,TOP,TOP
